In [1]:
import pandas as pd

In [2]:
df_novel = pd.read_csv('novel_story_sep.csv', lineterminator='\n')
df_sing = pd.read_csv('songChart_sep.csv', lineterminator='\n')

In [3]:
df_novel.columns

Index(['itemId', 'title', 'story', 'review', 'piece'], dtype='object')

In [4]:
df_sing.columns

Index(['id', 'year', 'songid', 'title', 'singer', 'lyrics', 'lyrics_sep'], dtype='object')

In [5]:
df_novel['all_story'] = df_novel['story'] + df_novel['review'] + df_novel['piece']
df_novel_new = df_novel[['title', 'all_story']]

In [6]:
df_sing['all_story'] = df_sing['lyrics_sep']
df_sing_new = df_sing[['title', 'all_story']]

In [7]:
df = pd.concat([df_novel_new, df_sing_new])

# LDA 모델에 들어갈 객체(dictionary, corpus) 만들고 학습시키기

In [8]:
import gensim
gensim.__version__

'3.8.1'

In [9]:
import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.test.utils import common_corpus
from gensim.models.wrappers import LdaMallet

In [10]:
novel = df_novel['story'] + df_novel['review'] + df_novel['piece']
sing = df_sing['lyrics_sep']

In [11]:
stories = novel.append(sing)

In [12]:
data_word = []
for story in stories:
    data = list(str(story).split())
    data_word.append(data)

In [13]:
id2word = corpora.Dictionary(data_word)

In [24]:
id2word.filter_extremes(no_below = 5) # 5회 이하로 등장한 단어는 삭제
texts = data_word
corpus = [id2word.doc2bow(text) for text in texts]

mallet_path = '../mallet-2.0.8/bin/mallet' 
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=60, random_seed=12, id2word=id2word)

coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=texts, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print(coherence_ldamallet) #응집성 지수

0.38603649556679703


In [25]:
# Select the model and print the topics
# optimal_model = model_list[model_list_num]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics()[5])

(38, '0.050*"청소년" + 0.030*"꿈" + 0.029*"학교" + 0.023*"자신" + 0.017*"선생님" + 0.014*"친구" + 0.010*"어른" + 0.010*"성장" + 0.009*"일" + 0.009*"현실"')


In [26]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    #ldamodel[corpus]: lda_model에 corpus를 넣어 각 토픽 당 확률을 알 수 있음
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num,topn=10)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
    print(type(sent_topics_df))

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#     sent_topics_df = pd.concat([Data['itemId'],Data['title'],Data['story'], sent_topics_df], axis=1)

    return(sent_topics_df)

In [27]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=texts, Data=df)

<class 'pandas.core.frame.DataFrame'>


In [28]:
# Format
df_topic_story = df_topic_sents_keywords

# 각 문서에 대한 토픽 내림차순
df_topic_sort=df_topic_story.sort_values(by=['Dominant_Topic'])

In [29]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)


In [30]:
sent_topics_sorteddf_mallet[['Perc_Contribution', 'Dominant_Topic', 'Topic_Keywords']]

,Perc_Contribution,Dominant_Topic,Topic_Keywords
87,0.6246,0.0,"면, 현실, 마음, 마법, 집, 동식, 나, 사람, 말, 큰"
9,0.4780,1.0,"서점, 섬, 유, 사진, 인생, 여인, 마치, 프랑스, 기욤, 뮈소"
881,0.3586,2.0,"죽음, 삶, 쪽, 죽은, 시작, 끝, 의미, 사고, 인생, 생"
1948,0.4449,3.0,"다시, 눈물, 가슴, 제발, 한번, 많이, 이별, 안녕, 이제, 다른"
834,0.4788,4.0,"사회, 인간, 세계, 세상, 시대, 자신, 삶, 노동, 자살, 현실"
459,0.4045,5.0,"시간, 황, 생각, 이서, 자신, 상점, 말, 미래, 발견, 기억"
510,0.8700,6.0,"말, 놈, 얼굴, 공격, 막, 노인, 화살, 죽, 상대, 무슨"
115,0.6415,7.0,"말, 사람, 세상, 나, 생각, 너, 약속, 줄, 수도, 살"
842,0.8215,8.0,"왕, 말, 눈, 혼, 조선, 황제, 너, 아비, 어찌, 그녀"
976,0.4769,9.0,"회사, 사람, 일, 배, 에피소드, 팀, 시간, 시장, 직원, 사장"


In [31]:
df_topic_story.value_counts('Dominant_Topic').sort_index()

Dominant_Topic
0.0      15
1.0       6
2.0      12
3.0     247
4.0      15
5.0      11
6.0       9
7.0       6
8.0      18
9.0       6
10.0     13
11.0     18
12.0     53
13.0     12
14.0     61
15.0     10
16.0     15
17.0     13
18.0     10
19.0     25
20.0     18
21.0      9
22.0     36
23.0     13
24.0     13
25.0      5
26.0     27
27.0     15
28.0      8
29.0      9
30.0    136
31.0     15
32.0      5
33.0      7
34.0      8
35.0     14
36.0     14
37.0     10
38.0     13
39.0     25
40.0      8
41.0     32
42.0      4
43.0    208
44.0     12
45.0     16
46.0     12
47.0     34
48.0      9
49.0    193
50.0     16
51.0     10
52.0     25
53.0    487
54.0      9
55.0     38
56.0     25
57.0     15
58.0     10
59.0      9
dtype: int64

In [36]:
topic_1 = df_topic_story[df_topic_story['Dominant_Topic']==1]
topic_2 = df_topic_story[df_topic_story['Dominant_Topic']==30]
topic_2[topic_2['Perc_Contribution'] > 0.4]

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
1610,30.0,0.4953,"오늘, 하루, 자꾸, 매일, 머리, 자꾸만, 어디, 느낌, 기분, 몰래","[자꾸, 두근두근, 가슴, 자꾸만, 자꾸만, 자꾸, 파오, 같이, 자꾸만, 자꾸만,..."
1765,30.0,0.4076,"오늘, 하루, 자꾸, 매일, 머리, 자꾸만, 어디, 느낌, 기분, 몰래","[월요일, 아마, 화요일, 보이지, 수요일, 뭔가, 느낌, 목요일, 그냥, 왠지, ..."
1838,30.0,0.4524,"오늘, 하루, 자꾸, 매일, 머리, 자꾸만, 어디, 느낌, 기분, 몰래","[취향, 저격, 취향, 저격, 느낌, 머리, 발끝, 취향, 저격, 가지, 안달, 자..."
1840,30.0,0.5506,"오늘, 하루, 자꾸, 매일, 머리, 자꾸만, 어디, 느낌, 기분, 몰래","[아예, 아예, 아예, 아예, 아예, 아예, 아예, 아예, 아예, 아예, 싱긋싱긋,..."
1847,30.0,0.4172,"오늘, 하루, 자꾸, 매일, 머리, 자꾸만, 어디, 느낌, 기분, 몰래","[월요일, 아마, 화요일, 보이지, 수요일, 뭔가, 느낌, 목요일, 그냥, 왠지, ..."


In [33]:
df_topic_sort[df_topic_sort['Perc_Contribution'] > 0.5]

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
23,0.0,0.6221,"면, 현실, 마음, 마법, 집, 동식, 나, 사람, 말, 큰","[당신, 감고, 시간, 있습니까, 위험한, 소원, 이루어지는, 위저드, 베이커, 리..."
87,0.0,0.6246,"면, 현실, 마음, 마법, 집, 동식, 나, 사람, 말, 큰","[당신, 감고, 시간, 있습니까, 판매, 위험한, 소원, 이루어지는, 위저드, 베이..."
510,6.0,0.8700,"말, 놈, 얼굴, 공격, 막, 노인, 화살, 죽, 상대, 무슨","[하북, 남쪽, 심주현, 위치, 유서, 깊은, 상계, 가문, 장씨, 남쪽, 중요, ..."
296,6.0,0.5015,"말, 놈, 얼굴, 공격, 막, 노인, 화살, 죽, 상대, 무슨","[하북, 남쪽, 심주현, 위치, 유서, 깊은, 상계, 가문, 장씨, 남쪽, 중요, ..."
513,6.0,0.8440,"말, 놈, 얼굴, 공격, 막, 노인, 화살, 죽, 상대, 무슨","[하북, 남쪽, 심주현, 위치, 유서, 깊은, 상계, 가문, 장씨, 남쪽, 중요, ..."
...,...,...,...,...
325,54.0,0.5382,"한국, 핀, 나, 부터, 월간, 결혼, 시, 유령, 서도, 임","[당대, 한국, 대적, 서도, 첨예한, 핀, 설선, 당대, 한국, 대적, 서도, 첨..."
747,54.0,0.5117,"한국, 핀, 나, 부터, 월간, 결혼, 시, 유령, 서도, 임","[당대, 한국, 대적, 서도, 첨예한, 핀, 설선, 서른, 당대, 한국, 대적, 서..."
1079,54.0,0.7447,"한국, 핀, 나, 부터, 월간, 결혼, 시, 유령, 서도, 임","[당대, 한국, 대적, 서도, 첨예한, 핀, 설선, 여섯, 대하, 여당, 한국, 대..."
1057,58.0,0.8120,"말, 명, 흰, 눈, 완성, 이순신, 길, 척, 싸움, 사람","[박은우, 장편, 명량, 불패, 신화, 명랑해전을, 승리, 이끌, 기, 악전, 고투..."
